In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Problem Set 3

## Øvelse 1: Implementer manuelt OLS-estimatoren

### Del 1: OLS i SLR-tilfældet (en enkelt forklarende variabel)


Når vi kun har 1 forklarende variabel og en konstant (SLR-tilfældet) arbejder vi med denne model:

$$ y_i = \beta_0 + \beta_1 x_i + u_i $$

Da ved vi fra den økonometriske teori at den sande værdi af $\beta_1$ er givet ved:

$$ \beta_1 = \frac{{\text{Cov}}(x, y)} {\text{Var}(x)}  $$

Vi ved også fra forelæsningerne at vi kan estimere covariansen og variansen sådan her:

\begin{align*}
\widehat{\text{Cov}}(x)  = \frac{1}{n} \sum (x_i - \bar{x})(y_i - \bar{y}) \\
\widehat{\text{Var}}(x) = \frac{1}{n} \sum (x_i - \bar{x})^2
\end{align*}

hvor $\bar{x}$ og $\bar{y}$ er de empiriske gennemsnit af $x, y$ i vores datasæt.  Sætter vi alt dette sammen ved hjælp af Analogi-princippet betyder det at vi kan estimere $\hat{\beta}_1$ med formlen:
$$
\hat{\beta}_1 = \frac{\sum (x_i - \bar{x})(y_i - \bar{y}) }{ \sum (x_i - \bar{x})^2}
$$

Dette er OLS-estimatoren for $\hat{\beta}_1$ i SLR-tilfældet.



#### Opgave 1 
Skriv en funktion `OLS_SLR()`, hvor du manuelt implementerer OLS-estimatoren for $\hat{\beta}_1$ i SLR-tilfældet. Brug numpy i din implementering. Funktionen skal tage x og y som argumenter, hvor x er et $n \times 1$ numpy array og y er et $n \times 1$ numpy array. Funktionen skal returnere skalar-værdien $\hat{\beta_1}$.

_Hints:_ 
- Du kan bruge numpy funktionen `np.mean()` til at beregne gennemsnittet af et numpy array og `np.sum()` til at beregne summen over et numpy array.



**Din kode:**

In [18]:
def OLS_SLR(x, y):
    return (np.sum( (x - np.mean(x)) * (y - np.mean(y)) )) /  np.sum((x - np.mean(x))**2)

#### Opgave 2
Her er noget meget simpelt testdata du kan bruge til at tjekke om din implementering er korrekt. Når du kører din funktion bør du få estimatet $\hat\beta_1 = 2.0$


In [36]:
X = np.array([ 1, 2, 3, 4, 5,])
Y = np.array([-4, 1, 3, 5, 4,])

print(OLS_SLR(X,Y))

(-4.2, 2.0, 1.98997487421324, 0.6)


#### Opgave 3 
Fra den økonometriske teori ved vi også, at vi kan estimere $\beta_0$ med formlen:
$$ \hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x} $$

Reflekter et øjeblik over formlen: Det giver intuitivt ret god mening. Estimatet af interceptet er gennemsnittet af vores $y$ observationer minus den estimerede effekt af den gennemsnitlige x-observation. 

Tegn et tilfældigt scatter plot på et stykke papir, tegn den bedste lige linje (en OLS-linje) igennem observationerne og se selv hvor den skærer y-aksen.

#### Opgave 4
Udvid din `OLS_SLR()` funktion så den returnerer ikke bare $\hat{\beta}_1$, men en tuple der indeholder ${(\hat\beta_0, \hat\beta_1)}$. Når du kører din udvidede funktion på samme testdata som før bør du få estimaterne ${(\hat\beta_0 = -4.2, \hat\beta_1 = 2.0)}$

**Din kode:**

In [21]:
def OLS_SLR(x, y):
    beta_1 = (np.sum( (x - np.mean(x)) * (y - np.mean(y)) )) /  np.sum((x - np.mean(x))**2)
    beta_0 = np.mean(y) - beta_1 * np.mean(x)
    return (beta_0, beta_1)

print(OLS_SLR(X,Y))

(-4.2, 2.0)


#### Opgave 5

Det sidste trin er at beregne standardfejlene for vores OLS-estimater.  For at gøre dette skal vi først beregne residualerne med formlen:
$$ \hat{u_i} = y - \beta_0 - \beta_1 x $$

Vi bruger residualerne til at estimere residualvariansen:

$$
\hat{\sigma}^2 = \frac{1}{n-2} \sum_{i=1}^{n} \hat{u}_i^2
$$

som vi kan bruge til at beregne variansen af OLS-estimatorerne:
$$
\text{Var}(\hat{\beta}_1 \mid X) = \frac{\hat{\sigma}^2}{\sum_{i=1}^{n} (x_i - \bar{x})^2}
$$
$$
\text{Var}(\hat{\beta}_0 \mid X) = \frac{\hat{\sigma}^2 \cdot \frac{1}{n}\sum_{i=1}^{n}x_{i}^{2}}{\sum_{i=1}^{n}(x_i - \bar{x})^2}
$$

Standardfejlene er ganske enkelt kvadratrødderne af varianserne:
$$
\text{se}(\hat{\beta}_1) = \sqrt{\text{Var}(\hat{\beta}_1 \mid X)}
$$
$$
\text{se}(\hat{\beta}_0) = \sqrt{\text{Var}(\hat{\beta}_0 \mid X)}
$$




Udvid din `OLS_SLR()` funktion så den også returnerer standardfejlene af koefficienterne. Når du kører funktionen på samme testdata som før skulle du gerne få:

$$ \hat \beta_0 = -4.2, \quad \hat \beta_1 = 2.0, \quad \text{se}(\hat \beta_0) = 1.99, \quad \text{se}(\hat \beta_1) = 0.6  $$

_Hint_: Brug funktionen `np.sqrt()` til at tage kvadratroden af et tal.

**Din kode:**

In [37]:
def OLS_SLR(x, y):
    beta_1 = (np.sum( (x - np.mean(x)) * (y - np.mean(y)) )) /  np.sum((x - np.mean(x))**2)
    beta_0 = np.mean(y) - beta_1 * np.mean(x)
    u_hat = y - beta_0 - beta_1 * x
    sigma2 = np.sum(u_hat**2) / (len(y) - 2)
    var_beta_1 = sigma2 / np.sum((x - np.mean(x))**2)
    var_beta_0 = (sigma2 * (np.sum(x**2))/len(y)) / np.sum((x - np.mean(x))**2)
    se_beta_1 = np.sqrt(var_beta_1)
    se_beta_0 = np.sqrt(var_beta_0)
    return (beta_0, beta_1, se_beta_0, se_beta_1)

print(OLS_SLR(X, Y))

(-4.2, 2.0, 1.98997487421324, 0.6)


#### Opgave 6 
Brug `sm.OLS()` funktionen fra statsmodels-pakken til at estimere samme model på testdataen og bekræft at dine koefficienter og standardfejl er de samme som dem statsmodels returnerer.

**Din kode:**

In [27]:
print(sm.OLS(Y, sm.add_constant(X)).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     11.11
Date:                Fri, 12 Sep 2025   Prob (F-statistic):             0.0446
Time:                        09:08:10   Log-Likelihood:                -9.0200
No. Observations:                   5   AIC:                             22.04
Df Residuals:                       3   BIC:                             21.26
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2000      1.990     -2.111      0.1

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "



### Del 2: OLS i MLR-tilfældet (Når vi har flere forklarende variable)


I det generelle tilfælde hvor vi har flere forklarende variable (MLR), ser modellen sådan her ud:

$$y_i = \beta_0 + \beta_1 x_{1 i}+ \beta_2 x_{2 i} +  \dots + \beta_k x_{k i} + u_i$$

Da ved vi at OLS-estimatoren kan skrives på matrix-form på følgende måde:

$$ \boldsymbol{\hat {\beta}} = (\mathbf{X'} \mathbf X)^{-1} \mathbf {X'} \mathbf y $$

hvor:
- $\boldsymbol{\hat{\beta}}$ er en $ (k+1) \times 1 $ vektor af koefficient estimater.
- $\mathbf X$ er en $ n \times (k+1) $ matrix indeholdende vores observationer af forklarende variable, hvor den første søjle er fyldt med 1-taller for at fange konstante effekter.
- $\mathbf y$ er en $ n \times 1 $ vektor som indeholder vores observationer af de afhængige variable

$n$ er antallet af observationer. $k$ er antallet af forklarende variable (vi plusser 1 for at gøre plads til konstantleddet).



#### Opgave 1
Skriv en funktion `OLS()`, hvor du manuelt implementerer OLS estimatoren i MLR-tilfældet ved hjælp af numpy. Funktionen skal tage X og Y som argumenter, hvor X er et $n \times (k+1)$ numpy array og Y er et $n \times 1$ numpy array. Funktionen skal returnere et $(k+1) \times 1$ numpy array $\hat{\beta}$.

_Hints:_ 
- Du kan gange matricer sammen i numpy ved hjælp af @ operatoren. Hvis du for eksempel har to kompatible matricer A og B, kan du bruge koden `A @ B` til at beregne matrix produktet. @ operatoren fungerer også til matrix-vector produkter.

- Du kan transponere en matrix i numpy ved at tilføje `.T` til sidst. For eksempel returnerer `A.T` den transpose af matricen A.
- Du kan invertere en matrix med `np.linalg.inv()` metoden. For eksempel returnerer `np.linalg.inv(A)` den inverse af matricen A.

**Din kode:** 

In [38]:
def OLS(X, Y):
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX)
    XtY = X.T @ Y
    beta_hat = XtX_inv @ XtY
    return beta_hat



#### Opgave 2 
Tjek at `OLS()` funktionen returnerer de samme koefficienter som i SLR-tilfældet når du kører den på samme data.

**Din kode:** 

In [39]:
print(OLS(np.column_stack((np.ones(len(X)), X)), Y))

[-4.2  2. ]


#### Opgave 3 
Lad os nu tjekke om funktionen også virker når vi har flere forklarende variable (flere søjler i X). Hvis din implementering er rigtig bør du få  følgende koefficient vektor $\hat{\beta}$: `array([-3.58043168,  1.61631032,  0.30264712])` når du kører nedenstående celle.

In [40]:
X = np.array([[ 1, 2, 3, 4, 5, 1, 0, 1],
              [-2, 5, 2, 1, 5, 7, 5, 2]]).T
X = sm.add_constant(X)
Y = np.array([-4, 1, 3, 5, 4, 1, -2.6, -1])

OLS(X,Y)

array([-3.58043168,  1.61631032,  0.30264712])

## Øvelse 2: SLR, MLR og makrodata
I denne opgave skal vi øve os i at formulere nul- og alternativhypoteser. Samtidig skal vi sammenligne en simpel regressionsmodel (SLR) med en multipel regressionsmodel (MLR).

Specifikt vil vi kigge på de makroøkonomiske hypoteser om om _absolut_ og _betinget_ konvergens. Teorierne omhandler langsigtet økonomisk vækst, og om lande kan vokse ud af fattigdom uden indgreb. 

- Teorien om **absolut konvergens** siger, at på lang sigt vil BNP pr. arbejder konvergere mod samme vækstbane. Det betyder, at jo længere væk et lands initiale BNP pr. arbejder er fra den langsigtede vækstbane, desto hurtigere vil BNP pr. arbejder vokse for at nå samme niveau. 

- Teorien om **betinget konvergens** siger, at et lands BNP pr. arbejder konvergerer mod en lande-specifik vækstbane, som afhænger af landets strukturelle karakteristika (fx befolkningstilvæksten og inveresteringsraten i fysisk kapital, som kan variere fra land til land)

Vi bruger datasættet PS3.dta, som indholder vækstdata for 93 lande fra 1960-2003. Dataen stammer fra Penn World Tables.

### Del 1: Simpel regressionsmodel (SLR)
I første omgang vil vi arbejde med følgende simple regressionsmodel:

$$ \text{gy}_i = \beta_0 + \beta_1 \ln(\text{y60}_i) + u_i $$

hvor:
- $\text{gy}_i $ er den gennemsnitlige årlige vækstrate i BNP pr. arbejder for land $i$ siden 1960
- $\text{y60}_i$ er landets BNP pr. arbejder i år 1960.

Bemærk, at vi ligesom i sidste uge tager log-transformationen af BNP pr. arbejder i 1960 for at opnå bedre numeriske resultater, og for at få en procentuel fortolkning af koefficient-estimatet.

#### Opgave 1
Hvad er nul- og alternativ hypotesen, når I gerne vil teste hypotesen om absolut konvergens? Vær præcis, når I formulerer nul- og alternativ hypotesen i ord.

**Dit svar:**


#### Opgave 2
Indlæs datasættet PS3.dta og drop de observationer som har datakvalitet D i kolonnen "grade". Hvilke lande bliver fravalgt? Hvilken betydning for analysen om absolut konvergens kan det have, at vi frasorterer lande med lav datakvalitet?

**Din kode:**


**Dit svar:**


#### Opgave 3
Konstruer variablen $\ln (\text{y60}_i)$ og estimer den simple regressionsmodel med `sm.OLS()`.  Hvad er modellens forklaringsgrad, $R^2$? Hvad kan I konkludere om absolut konvergens?

**Din kode:**

**Dit svar:**

### Del 2: Multipel regressionsmodel
Vi udvider nu modellen til at inkludere en variabel, som tager højde for forskelle i strukturelle karakteristika på tværs af lande:

$$ \text{gy}_i = \beta_0 + \beta_1 \ln(\text{y60}_i) + \beta_2 \text{struc}_i + u_i $$

hvor de strukturelle karakteristika er defineret på følgende måde:

$$ 
\text{struc}_i = \ln(\text{sk}_i) − \ln(\text{n}_i + 0, 075)
$$ 

Da I endnu ikke har haft Makroøkonomi A behøver I ikke forstå modellen fuldt ud, men det kan nævnes at:

- $\text{sk}_i$ er investeringsraten i fysisk kapital for land $i$

- $\text{n}_i$ er den gennemsnitlige årlige befolkningsvækst i perioden 1960-2003

- De 0,075 i det sidste led er et estimat for afskrivningsraten af fysisk kapital i den basale Solow model. Bemærk, at det er samme værdi for alle lande. I lærer mere om alt dette i Makro A.

#### Opgave 4
Hvad er nul- og alternativhypotesen, når vi gerne vil teste hypotesen om _betinget_ konvergens?

**Dit svar:** 

#### Opgave 5
Konstruer den nye variabel $\text{struc}_i$. Estimer den multiple regressionsmodel. Hvor meget af variationen i den afhængige variable er forklaret af modellen? Fortolk koefficienten for det initiale niveau af BNP pr. capita. Hvad kan vi konkludere om betinget konvergens? Overvej om en anden afskrivningsrate vil have betydning for jeres estimater.

**Din kode:**

**Dit svar:**

#### Opgave 6
I denne opgave vil du lære hvordan du let kan eksportere output fra statsmodels og fra pandas DataFrames til LaTeX, sådan at du kan bruge output fra Python i pæne tabeller, der er korrekt sat op i dine afleveringsopgaver.


- Brug denne kommando i Python til at printe dit regression summary som LaTeX kode:
```py
print(results.summary().as_latex())
```

- Opret et LaTeX-dokument (fx i Overleaf).

- Kopier koden ind i dit LaTeX dokument og se, hvordan tabellen ser ud.

Tilsvarende kan du konvertere ethvert Pandas DataFrame (fx deskriptiv statistik) til LaTeX med metoden `.to_latex()`. Bemærk at navngivningen af de to metoder er lidt forskellig i statsmodels og i pandas.

Prøv fx at køre koden:
```py
print(df.describe().to_latex())
```

*Tip:* Hvis du vil afrunde dit LaTeX output til fx to decimaler, kan du bruge argumentet `float_format="%.2f"` i `.to_latex()` metoden

*Tip:* Din tabel bliver automatisk flottere i LaTeX hvis du importerer LaTeX-pakken booktabs:
```\usepackage{booktabs}```

**Din kode:**

Du vil måske opleve, at det kræver lidt mere arbejde at få tabellen til at se godt ud i LaTeX. Blandt andet fordi statsmodels faktisk spytter flere tabeller af forskellig størrelse ud, når man printer et summary.

Du kan bede statsmodels om at printe tabellerne enkeltvist sådan her her. Så kan du fx vælge kun at bruge tabellen der indeholder koefficientestimaterne.

```py
print(results.summary().tables[0])
print(results.summary().tables[1])
print(results.summary().tables[2])
```

**Sidste tip til de "dovne":** Hvis du har travlt og ikke har lyst til at rode med at sætte LaTeX-tabellerne pænt op, kan du også bare kopiere de rå tekstoutput fra Python og indsætte i et verbatim miljø i LaTeX – det ser OK ud. Du skriver følgende i LaTeX:

```LaTeX
\begin{verbatim}
<Dit rå textoutput her>
\end{verbatim}
```